In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
df_sentencias = pd.read_json('xxxxxxxxxxxxxxxxxxxxxx.jsonl',lines=True)
df_sentencias['bias'] = [1 if len(x) > 0 else 0 for x in df_sentencias['highlight']]
df_sentencias = df_sentencias[~df_sentencias['doc'].str.startswith('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')]
aa = df_sentencias.groupby('doc')[['bias']].sum()
df_sentencias = df_sentencias[df_sentencias['doc'].isin(aa[aa['bias'] > 0].index)]
df_sentencias

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def compute_metrics(pred,y_test):
    metrics = {}

#     print(pred)
#     print(y_test)
#     print('-----------------------')

    metrics['accuracy'] = accuracy_score(y_test,pred)

    metrics['precision'] = precision_score(y_test,pred,average='binary')
    metrics['recall'] = recall_score(y_test,pred,average='binary')
    metrics['fmeasure'] = f1_score(y_test,pred,average='binary')

    metrics['precision_weighted'] = precision_score(y_test,pred,average='weighted')
    metrics['recall_weighted'] = recall_score(y_test,pred,average='weighted')
    metrics['fmeasure_weighted'] = f1_score(y_test,pred,average='weighted')
    metrics['average_precision_score_weighted'] = average_precision_score(y_test,pred,average='weighted')
    if len(set(y_test)) != 1:
        metrics['roc_auc_score_weighted'] = roc_auc_score(y_test,pred,average='weighted')

    metrics['precision_micro'] = precision_score(y_test,pred,average='micro')
    metrics['recall_micro'] = recall_score(y_test,pred,average='micro')
    metrics['fmeasure_micro'] = f1_score(y_test,pred,average='micro')
    metrics['average_precision_score_micro'] = average_precision_score(y_test,pred,average='micro')
    if len(set(y_test)) != 1:
        metrics['roc_auc_score_micro'] = roc_auc_score(y_test,pred,average='micro')

    metrics['precision_macro'] = precision_score(y_test,pred,average='macro')
    metrics['recall_macro'] = recall_score(y_test,pred,average='macro')
    metrics['fmeasure_macro'] = f1_score(y_test,pred,average='macro')
    metrics['average_precision_score_macro'] = average_precision_score(y_test,pred,average='macro')
    if len(set(y_test)) != 1:
        metrics['roc_auc_score_macro'] = roc_auc_score(y_test,pred,average='macro')

    metrics['matthews_corrcoef'] = matthews_corrcoef(y_test,pred) # 0 es random prediction, 1 es perfecto, -1 inverse prediction

    metrics['y_reales'] = sum(y_test)
    metrics['y_pred'] = sum(pred)
    metrics['total'] = len(pred)

    return metrics

In [ ]:
import json
from json import JSONDecodeError
import pickle
import os

path_dir = 'results/'

import re
re_json = re.compile('\{.*\}')

listi_metrics = []

from collections import defaultdict
categories = defaultdict(dict)

for ff in os.listdir(path_dir):

    if not ff.startswith('responses'):
        continue

    with open(path_dir + ff,'rb') as file:
        dd = pickle.load(file)

    print('-----------------------------',ff)

    listi = []
    for x in dd:
        cc = str(x[1] if isinstance(x[1],str) else x[1].content).replace('\n',' ')
#         print(cc)
        cc = re_json.search(cc)
#         print(cc)
        if cc is None:
            continue
        cc = cc.group().replace('{\'','{"').replace('\'}','"}').replace('\':','":').replace(':\'',':"').replace(': \'',':"').replace('`','"').replace('""','"').replace('""','"')
#         .replace(' \'','"').replace('\' ','"')
        cc = re.sub('\s+',' ',cc).replace('" "','", "').replace('". "','", "').replace('"; "','", "')
        try:
            cc = json.loads(cc)
        except JSONDecodeError as e:
            print(cc)
            print('_________________________',e)
            cc = None

        if cc is None:
            continue
        ll = None
        if 'label' in cc: ll = 'label'
        elif 'categoria' in cc: ll = 'categoria'
        elif 'categoría' in cc: ll = 'categoría'
        elif 'categorias' in cc: ll = 'categorias'
        elif 'categorías' in cc: ll = 'categorías'
        elif 'category' in cc: ll = 'category'
        elif 'sesgo' in cc: ll = 'sesgo'
        else:
            print(df_sentencias[df_sentencias.index == x[0]]['bias'].values[0],'--',cc.keys())
            continue

        if ll is None:
            continue

        if isinstance(cc[ll],bool):
            cc[ll] = 'sesgo' if cc[ll] else 'no-sesgo'

        if not isinstance(cc[ll],str):
            continue

#         print(cc[ll])
        cc = 1 if (cc[ll].lower() == 'sesgo' or cc[ll].lower() == 'bias') else 0 if (cc[ll].lower() == 'no-sesgo' or cc[ll].lower() == 'no-bias') else -1
        if cc == -1:
            continue
        listi.append({'index':x[0],'y_pred':cc})
#     print(listi)
    df_pred = pd.DataFrame(listi).set_index('index')

    df_merge = df_sentencias.merge(df_pred,left_index=True,right_index=True)
    metrics = compute_metrics(df_merge['y_pred'],df_merge['bias'])
    if 'fold' in ff:
        metrics['which'] = '_'.join(ff.split('_')[0:-1])
        metrics['fold'] = int(ff[-1])
    else:
        metrics['which'] = ff

    categories[metrics['which']].update(df_merge[['y_pred']].to_dict()['y_pred'])
    listi_metrics.append(metrics)
#     print()
#     print(metrics)
#     print()

dd = pd.DataFrame(listi_metrics)
dd.to_csv('__results_folds.csv')
ee = dd.groupby('which').mean().sort_values('precision',ascending=False)
ee.to_csv('__results_folds_avg.csv')
ee

In [ ]:
# dd.sort_values('precision',ascending=False)
df_categories = pd.DataFrame.from_dict(categories)
df_categories = df_categories.join(df_sentencias[['bias']])

ll = []
for i in tqdm(range(0,len(df_categories.columns))):
    x = df_categories.columns[i]
    for j in range(i+1,len(df_categories.columns)):
        y = df_categories.columns[j]
        if x == 'bias' or y == 'bias':
            continue
        aa = [1 if z[0] == 1 and z[1] == 1 else 0 for z in zip(df_categories[x],df_categories[y])]
        mm = compute_metrics(aa,df_categories['bias'])
        mm['which_A'] = x
        mm['which_B'] = y
        ll.append(mm)

pd.DataFrame(ll).sort_values('recall',ascending=False)

In [ ]:
import scipy
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon

from statistics import mean, stdev
from math import sqrt

def cohens_d(c0,c1):
    den = sqrt((stdev(c0) ** 2 + stdev(c1) ** 2) / 2)
    if den == 0:
        return '-'
    cohens_d = abs((mean(c0) - mean(c1)) / den)
    if cohens_d < 0.1:
        return 'negligible'
    if cohens_d < 0.35:
        return 'small'
    if cohens_d < 0.65:
        return 'medium'
    return 'large'

def paired_analysis(f1,f2,alpha=0.01): # only one df is needed, then is for i for j

    if len(f1) != len(f2) or len(f1) == 1:
        return None

    normal = False
    try:
         if scipy.stats.normaltest(f1,nan_policy='omit').pvalue < alpha and scipy.stats.normaltest(f2,nan_policy='omit').pvalue < alpha:
                normal = True
    except:
        pass
    if not normal:
        statsb = wilcoxon(f1,f2,alternative='two-sided',zero_method='zsplit')
        statsg = wilcoxon(f1,f2,alternative='greater',zero_method='zsplit')
        statsl = wilcoxon(f1,f2,alternative='less',zero_method='zsplit')
        return (statsb.pvalue,statsg.pvalue,statsl.pvalue,cohens_d(f1,f2))
    else: # both normal
        statsb = scipy.stats.ttest_rel(f1,f2,nan_policy='omit')
#         print(statsb)
        return (statsb.statistic,statsb.pvalue,cohens_d(f1,f2))

    return differents

In [ ]:
import warnings
warnings.filterwarnings("ignore")

ee = dd[dd['fold'] == dd['fold']]
ee = ee.drop(columns=['y_reales','y_pred','total','fold'])
ee_list = list(ee['which'].unique())

listi_metrics = []
alpha = 0.01

for x in tqdm(ee_list):

    ee_x = ee[ee['which'] == x].drop(columns=['which'])

    for y in ee_list:
        if x == y:
            continue

        ee_y =  ee[ee['which'] == y].drop(columns=['which'])


        dicti = {}
        dicti['X'] = x
        dicti['Y'] = y

#         print(x,y)

        for c in ee_x.columns:

            stats = paired_analysis(ee_x[c],ee_y[c],alpha)
            if stats is None:
                continue
            if len(stats) < 4: # normal -- tiene que dividir y chequear para ver cual queremos rejectear
                dicti[c] = stats[2] if stats[1]/2 < alpha and stats[0] > 0 else '-'
            else:
                dicti[c] = stats[3] if stats[1] < alpha else '-'

        if len(dicti) != 2:
            listi_metrics.append(dicti)

df_stats = pd.DataFrame(listi_metrics)
df_stats.to_csv('df_stats_fold.csv',index=False)
df_stats

In [ ]:
# definir ff para juntar todas las combinaciones !
ff = pd.read_csv('all_best_metrics_df.csv',sep=';')
ff = pd.concat([ff,pd.read_csv('__results_folds_avg.csv')])
ff = ff.dropna(axis=1).set_index('which').T
ff

In [ ]:
# ff = dd.groupby('which').mean().sort_values('precision',ascending=False).drop(columns=['y_reales','y_pred','total','fold']).T

alpha = 0.01
listi = []
for x in tqdm(ff.columns):
    for y in ff.columns:
        if x == y:
            continue
        stats = paired_analysis(ff[x],ff[y],alpha)
        if stats is None:
            continue

        dicti = {}
        dicti['X'] = x
        dicti['Y'] = y
        if len(stats) < 4: # normal -- tiene que dividir y chequear para ver cual queremos rejectear
            z = stats[2] if stats[1]/2 < alpha and stats[0] > 0 else '-'
        else:
            z = stats[3] if stats[1] < alpha else '-'

        dicti['Z'] = z
        listi.append(dicti)

df_stats_full = pd.DataFrame(listi)
df_stats_full.to_csv('df_stats_full.csv',index=False)
df_stats_full

In [ ]:
import ranky

In [ ]:
def get_best_params_(df_combs,metric=None,top=15):
    if metric is not None:
        if 'user_attrs_metric' in df_combs.columns:
            model = df_combs[df_combs['user_attrs_metric'] == metric]
        else:
            model = df_combs[df_combs['target_metric'] == metric]
    else:
        model = df_combs

    if 'user_attrs_1-precision' in df_combs.columns:
        metrics_to_rank = ['user_attrs_1-precision','user_attrs_1-recall','user_attrs_metric']
    else:
        metrics_to_rank = ['1.precision','1.recall','mcc']
    model_metrics = model[metrics_to_rank]
    rank = ranky.pairwise(model_metrics).to_dict()
    rank = [(k, v) for k, v in rank.items()]
    rank.sort(key=lambda x: -x[1])
    best_params_top = []
    for i in range(0,min(top,len(rank))):
        best_params_top.append(model[model.index == rank[i][0]].to_dict('records')[0])
    return pd.DataFrame(best_params_top)

In [ ]:
def get_best_params(path,top=15,metrics=None):

    if metrics is None:
        metrics = ['PRAUC','Precision','Recall','MCC']

    df_combs = pd.read_csv(path)
    if 'user_attrs_1-precision' in df_combs.columns:
        df_combs['user_attrs_1-fmeasure'] = [(2*df_combs['user_attrs_1-precision'].values[i] * df_combs['user_attrs_1-recall'].values[i])/(df_combs['user_attrs_1-precision'].values[i] + df_combs['user_attrs_1-recall'].values[i]) for i in range(0,len(df_combs))]
        df_combs.sort_values('user_attrs_1-fmeasure',ascending=False)

    ll = {}
    for m in metrics:
        print(path, '::',m)
        ll[m] = get_best_params_(df_combs,metric=m,top=top)

    return ll

In [ ]:
best_pp = {}
# best_pp['bertlegal4'] = get_best_params('bertlegal4-studies_df.csv')
# best_pp['multilingua4'] = get_best_params('multilingua4-studies_df.csv')
# best_pp['beto4'] = get_best_params('beto4-studies_df.csv')
# best_pp['tfidf'] = get_best_params('tfidf-studies_df.csv')

best_pp['multilingua4'] = get_best_params('multilingua-all_best_metrics_df.csv')
best_pp['bertlegal4'] = get_best_params('bertlegal-all_best_metrics_df.csv')

In [ ]:
alldd = pd.DataFrame()
for k in best_pp:
    bp = best_pp[k]
    dd = pd.DataFrame()
    for m in bp:
        bp[m]['which'] = k
        dd = pd.concat([dd,bp[m]])
    if 'user_attrs_1-fmeasure' in dd.columns:
        dd = dd.sort_values(by=['user_attrs_1-fmeasure'],ascending=False)#[20:35]
    else:
        dd = dd.sort_values(by=['1.f1-score'],ascending=False)#[20:35]
    alldd = pd.concat([alldd,dd])
alldd

In [ ]:
# alldd.sort_values(['user_attrs_1-precision','user_attrs_1-recall'],ascending=False)
# alldd.sort_values(['user_attrs_1-fmeasure'],ascending=False).to_csv('best_params_tfidf.csv',index=False)
alldd.to_csv('best_params_legal_multi.csv',index=False)

In [ ]:
import json
from json import JSONDecodeError
import pickle
import os
from tqdm.notebook import tqdm

path_dir = '/results/'

import re
re_json = re.compile('\{.*\}')

listi = []
from collections import defaultdict
categories = defaultdict(dict)

for ff in tqdm(os.listdir(path_dir)):

    if not ff.startswith('responses'):
        continue

    if 'expli' not in ff: # solo los que pedimos que tuvieran
        continue

    print(ff)

    with open(path_dir + ff,'rb') as file:
        dd = pickle.load(file)

    for x in dd:
        cc = str(x[1] if isinstance(x[1],str) else x[1].content).replace('\n',' ')
#         print(cc)
        cc = re_json.search(cc)
#         print(cc)
        if cc is None:
            continue
        cc = cc.group().replace('{\'','{"').replace('\'}','"}').replace('\':','":').replace(':\'',':"').replace(': \'',':"').replace('`','"').replace('""','"').replace('""','"')
#         .replace(' \'','"').replace('\' ','"')
        cc = re.sub('\s+',' ',cc).replace('" "','", "').replace('". "','", "').replace('"; "','", "')
        try:
            cc = json.loads(cc)
        except JSONDecodeError as e:
#             print(cc)
#             print('_________________________',e)
            cc = None

        if cc is None:
            continue

        l = None # este es para todos
        if 'explicación' in cc: l = 'explicación'
        elif 'explanation' in cc: l = 'explanation'
        elif 'explicacion' in cc: l = 'explicacion'
        else:
            print(cc.keys())
            continue

        if l is None:
            continue

        ll = None
        if 'label' in cc: ll = 'label'
        elif 'categoria' in cc: ll = 'categoria'
        elif 'categoría' in cc: ll = 'categoría'
        elif 'categorias' in cc: ll = 'categorias'
        elif 'categorías' in cc: ll = 'categorías'
        elif 'category' in cc: ll = 'category'
        elif 'sesgo' in cc: ll = 'sesgo'

        if ll is None:
            continue
        if isinstance(cc[ll],bool):
            print(cc)
        ll = 1 if (isinstance(cc[ll],bool) and cc[ll]) else 0 if (isinstance(cc[ll],bool) and not cc[ll]) else 1 if (cc[ll].lower() == 'sesgo' or cc[ll].lower() == 'bias') else 0 if (cc[ll].lower() == 'no-sesgo' or cc[ll].lower() == 'no-bias') else -1
        if ll == -1:
            continue

        listi.append({'index':x[0],'text': df_sentencias[df_sentencias.index == x[0]]['text'].values[0], 'doc': df_sentencias[df_sentencias.index == x[0]]['doc'].values[0], 'label': df_sentencias[df_sentencias.index == x[0]]['bias'].values[0], 'explicación':(ll,cc[l]),'which':ff})

df_expli = pd.DataFrame(listi).pivot(columns='which',values='explicación',index=['index','text','label','doc'])
df_expli

In [ ]:
df_expli.reset_index().set_index('index').to_pickle('df_explicaciones.pickle')

In [ ]:
df_expli.columns

In [ ]:
aa = pd.read_csv('tfidf-studies_df.csv')
aa['user_attrs_1-fmeasure'] = [(2*aa['user_attrs_1-precision'].values[i] * aa['user_attrs_1-recall'].values[i])/(aa['user_attrs_1-precision'].values[i] + aa['user_attrs_1-recall'].values[i]) for i in range(0,len(aa))]
aa = aa.sort_values('user_attrs_1-fmeasure',ascending=False)
aa[aa['user_attrs_1-fmeasure'] < 0.45]

In [ ]:
# hacer el de popularidad
from sklearn.dummy import DummyClassifier

du = DummyClassifier(strategy='most_frequent')

bias = [1 if x == 0 else 0 for x in df_sentencias['bias']]

du.fit(df_sentencias['text'],bias)
compute_metrics(du.predict(df_sentencias['text']).tolist(),bias)

In [ ]:
# hacer la tabla para latex

df = pd.read_excel('results.xlsx',sheet_name='Traditional')
df = df[['which','precision','recall','precision_weighted','recall_weighted','matthews_corrcoef']]
df = df.dropna(how='all').dropna(axis=0)
df

out = []
out.append(' & '.join(df.columns))
for row in df.iterrows():
    ll = row[1].to_dict()
    ll = [v if isinstance(v,str) else str(round(v,3)) for k,v in ll.items()]
    out.append(' & '.join(ll) + ' \\tabularnewline')
print('\n'.join(out))
